<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/assignment/ass6/bdm/Thruth%20Seeker/Assignment_6_by_Thruth_Seeker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 6 by Thruth Seeker

| Name                                     | Matrix Number |
| :---------------------------------------- | :-------------: |
| Hazem Fenneer            |MCS231019      |
| Shivanesh A/L Sivakumar               |MCS231014      |

**Dealing with large datasets poses challenges in terms of memory efficiency and processing speed. In this exploration, we'll employ five smart strategies to effectively manage substantial datasets**

1. Load Less Data
2. Use Chunking
3. Optimize Data Types
4. Sampling
5. Parallelize with Dask

## Downloading the [Dataset](https://www.kaggle.com/datasets/rmjacobsen/property-listings-for-5-south-american-countries?select=ar_properties.csv)

First we Downlaod and unzip the dataset from keggal and make it availabe in the colab

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Assignment 2"

In [3]:
!kaggle datasets download -d rmjacobsen/property-listings-for-5-south-american-countries

 99% 469M/475M [00:08<00:00, 71.3MB/s]
100% 475M/475M [00:08<00:00, 59.0MB/s]


In [4]:
!unzip \*.zip  && rm *.zip

Archive:  property-listings-for-5-south-american-countries.zip
  inflating: ar_properties.csv       
  inflating: co_properties.csv       
  inflating: ec_properties.csv       
  inflating: pe_properties.csv       
  inflating: uy_properties.csv       


In [ ]:
!ls

ar_properties.csv  gdrive						 sample_data
co_properties.csv  pe_properties.csv					 uy_properties.csv
ec_properties.csv  property-listings-for-5-south-american-countries.zip


In [5]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import psutil
import time
import os
file_path='/content/ar_properties.csv'


Here, we called all the necessary libraries.

##First: the Traditional Method (pandas)

In [47]:
file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convert to megabytes

df = pd.read_csv(file_path)

In [68]:
df.head(10)

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,2019-09-15,-26.815439,-65.312393,Argentina,Tucumán,Yerba Buena,...,NaN,NaN,NaN,NaN,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,2019-09-15,-26.839469,-65.212790,Argentina,Tucumán,NaN,...,1.0,55.0,41.0,NaN,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.424820,-64.181225,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.421242,-64.190798,Argentina,Córdoba,Córdoba,...,1.0,48.0,48.0,NaN,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.426064,-64.180042,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
5,vY1UjrDt8jFIGPtJfdpNZQ==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.429089,-64.189291,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,Departamento - Nueva Cordoba,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
6,D1KQkn+XsaOHRKVUuQRtpg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.418640,-64.188257,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Vélez Sarsfield al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
7,2yVnYhBoYRQ5r/mfQ32Mwg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.425732,-64.180824,Argentina,Córdoba,Córdoba,...,1.0,40.0,40.0,NaN,NaN,Mensual,Departamento - Nueva Cordoba,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
8,t0kNTwjTWOdat8Z87X7QLw==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.423450,-64.184201,Argentina,Córdoba,Córdoba,...,NaN,55.0,55.0,NaN,NaN,Mensual,1 DORMITORIO | Ituzaingó al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
9,fY4sOgwvFSS5Qz2M66X7/A==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.423824,-64.181516,Argentina,Córdoba,Córdoba,...,1.0,44.0,44.0,NaN,NaN,Mensual,1 DORMITORIO | Obispo Salguero al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal


In this example, we employed traditional data handling approaches without incorporating any advanced methods.

Data processing

In [69]:
df.size

25000000

In [70]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')

In [71]:
df.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
              'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
              'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
              'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']


In [ ]:
df.columns

Index(['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude',
       'Longitude', 'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4',
       'L5', 'L6', 'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area',
       'Covered_Area', 'Price', 'Currency', 'Price_Period', 'Title',
       'Description', 'Property_Type', 'Operation_Type'],
      dtype='object')

In [ ]:
df.shape

(1000000, 25)

There are i million rows and 25 columns in this dataset.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   ID                1000000 non-null  object 
 1   Ad_Type           1000000 non-null  object 
 2   Start_Date        1000000 non-null  object 
 3   End_Date          1000000 non-null  object 
 4   Created_On        1000000 non-null  object 
 5   Latitude          850472 non-null   float64
 6   Longitude         849456 non-null   float64
 7   Country_L1        1000000 non-null  object 
 8   Province_L2       1000000 non-null  object 
 9   City_L3           944673 non-null   object 
 10  Neighbourhood_L4  249223 non-null   object 
 11  L5                4616 non-null     object 
 12  L6                0 non-null        float64
 13  Rooms             531101 non-null   float64
 14  Bedrooms          368640 non-null   float64
 15  Bathrooms         770378 non-null   float64
 16  T

In [72]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
df = df.drop_duplicates()
result = df.count()

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

Execution Time: 6.038107395172119 seconds
Memory Usage Before: 8251.4375 MB
Memory Usage After: 8456.59765625 MB
Memory Usage Difference: 205.16015625 MB


1. The operation took approximately 7.04 seconds to complete.
2. The memory usage before the operation was 1441.72 megabytes (MB).
3. The memory usage after the operation increased to 1656.56 megabytes (MB).
4. The difference in memory usage (after - before) is 214.84 megabytes (MB). This represents the additional memory consumed during the execution of the operation.

In [73]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
null_counts = df.isnull().sum()

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

# Display the result of df.isnull().sum()
print("Null Counts:")
print(null_counts)

Execution Time: 2.37729811668396 seconds
Memory Usage Before: 8456.59765625 MB
Memory Usage After: 8466.85546875 MB
Memory Usage Difference: 10.2578125 MB
Null Counts:
ID                        0
Ad_Type                   0
Start_Date                0
End_Date                  0
Created_On                0
Latitude             149528
Longitude            150544
Country_L1                0
Province_L2               0
City_L3               55327
Neighbourhood_L4     750777
L5                   995384
L6                  1000000
Rooms                468899
Bedrooms             631360
Bathrooms            229622
Total_Area           421223
Covered_Area         450055
Price                 48673
Currency              54505
Price_Period         581101
Title                    28
Description              25
Property_Type             0
Operation_Type            0
dtype: int64


In [74]:
# l5 and L6 are not clear, so drop them
df = df.drop(['L5', 'L6'], axis=1)


In [75]:
#Impute Categorical Values:
df['City_L3'].fillna(df['City_L3'].mode()[0], inplace=True)
df['Neighbourhood_L4'].fillna(df['Neighbourhood_L4'].mode()[0], inplace=True)
df['Price_Period'].fillna(df['Price_Period'].mode()[0], inplace=True)


In [76]:
#text
df['Title'].fillna('', inplace=True)
df['Description'].fillna('', inplace=True)


In [77]:
#Impute Numerical Values
df['Latitude'].fillna(df['Latitude'].mean(), inplace=True)
df['Longitude'].fillna(df['Longitude'].mean(), inplace=True)
df['Rooms'].fillna(df['Rooms'].mean(), inplace=True)
df['Bedrooms'].fillna(df['Bedrooms'].mean(), inplace=True)
df['Bathrooms'].fillna(df['Bathrooms'].mean(), inplace=True)
df['Total_Area'].fillna(df['Total_Area'].mean(), inplace=True)
df['Covered_Area'].fillna(df['Covered_Area'].mean(), inplace=True)
df['Price'].fillna(df['Price'].mean(), inplace=True)

In [78]:
df.isnull().sum()

ID                      0
Ad_Type                 0
Start_Date              0
End_Date                0
Created_On              0
Latitude                0
Longitude               0
Country_L1              0
Province_L2             0
City_L3                 0
Neighbourhood_L4        0
Rooms                   0
Bedrooms                0
Bathrooms               0
Total_Area              0
Covered_Area            0
Price                   0
Currency            54505
Price_Period            0
Title                   0
Description             0
Property_Type           0
Operation_Type          0
dtype: int64

for now, lets keep Currency

In [79]:
print(df['ID'].nunique())
print(df['Ad_Type'].nunique())
print(df['Start_Date'].nunique())
print(df['End_Date'].nunique())
print(df['Created_On'].nunique())
print(df['Latitude'].nunique())
print(df['Longitude'].nunique())
print(df['Country_L1'].nunique())
print(df['Province_L2'].nunique())
print(df['City_L3'].nunique())
print(df['Neighbourhood_L4'].nunique())
print(df['Rooms'].nunique())
print(df['Bedrooms'].nunique())
print(df['Bathrooms'].nunique())
print(df['Total_Area'].nunique())
print(df['Covered_Area'].nunique())
print(df['Price'].nunique())
print(df['Currency'].nunique())
print(df['Price_Period'].nunique())
print(df['Title'].nunique())
print(df['Description'].nunique())
print(df['Property_Type'].nunique())
print(df['Operation_Type'].nunique())


1000000
1
368
448
368
330930
333351
4
42
1267
982
36
79
21
5448
2801
20033
4
3
497587
685520
10
3


In [ ]:
df.describe()

,Latitude,Longitude,Rooms,Bedrooms,Bathrooms,Total_Area,Covered_Area,Price
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1.000000e+06
mean,-34.669838,-59.333241,2.881469,2.098199,1.682252,483.990998,4.372907e+02,2.857189e+05
std,2.745181,2.532343,1.234549,1.516532,0.935525,3085.644863,3.750450e+04,1.135263e+07
min,-54.975338,-180.000000,1.000000,-13.000000,1.000000,-200.000000,-1.700000e+02,0.000000e+00
25%,-34.720683,-59.333241,2.881469,2.098199,1.000000,78.000000,6.500000e+01,3.500000e+04
50%,-34.620537,-58.512473,2.881469,2.098199,1.682252,483.990998,2.900000e+02,1.050000e+05
75%,-34.521630,-58.389603,3.000000,2.098199,2.000000,483.990998,4.372907e+02,2.500000e+05
max,85.051129,-4.496101,40.000000,390.000000,20.000000,200000.000000,2.410000e+07,1.000000e+10


##New Methods

###Load Less Data


In this, we will strategically load less data to optimize data usage.
we will measure the time and memory usage to see if there's any differences.

In [ ]:
selected_columns = ['id', 'ad_type', 'start_date', 'end_date', 'bathrooms',
                    'surface_total', 'surface_covered', 'price', 'currency',
                    'price_period', 'title', 'description', 'property_type', 'operation_type']
df_selected = pd.read_csv(file_path, usecols=selected_columns)

Performaing the same data preparation from df

In [ ]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

print(df_selected.head(10))
print(df_selected.size)
print(df_selected.columns)
print(df_selected.shape)
print(df_selected.info())
# Perform operations
df_selected = df_selected.drop_duplicates()
result = df_selected.count()

df_selected.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Bathrooms',
                       'Total_Area', 'Covered_Area', 'Price', 'Currency',
                       'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']
null_counts = df_selected.isnull().sum()

#Impute Categorical Values:
df_selected['Price_Period'].fillna(df_selected['Price_Period'].mode()[0], inplace=True)

#text
df_selected['Title'].fillna('', inplace=True)
df_selected['Description'].fillna('', inplace=True)

#Impute Numerical Values
df_selected['Bathrooms'].fillna(df_selected['Bathrooms'].mean(), inplace=True)
df_selected['Total_Area'].fillna(df_selected['Total_Area'].mean(), inplace=True)
df_selected['Covered_Area'].fillna(df_selected['Covered_Area'].mean(), inplace=True)
df_selected['Price'].fillna(df_selected['Price'].mean(), inplace=True)
df_selected.isnull().sum()

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

                         ID    Ad_Type  Start_Date    End_Date  Bathrooms  \
0  7LoZFkSIPOTox0r32ck42Q==  Propiedad  2019-09-15  2019-10-26   1.682252   
1  QsZD4OxZInNd5po5LQDRmg==  Propiedad  2019-09-15  9999-12-31   1.000000   
2  qDtysoUgbnHLp0W9We+8fg==  Propiedad  2019-09-15  2019-09-25   1.000000   
3  olj51zV0HFurmaZ78U0ssg==  Propiedad  2019-09-15  2019-09-25   1.000000   
4  OcS1SAA5oAzjZ3Mzg3XHyg==  Propiedad  2019-09-15  2019-09-25   1.000000   
5  vY1UjrDt8jFIGPtJfdpNZQ==  Propiedad  2019-09-15  2019-09-25   1.000000   
6  D1KQkn+XsaOHRKVUuQRtpg==  Propiedad  2019-09-15  2019-09-25   1.000000   
7  2yVnYhBoYRQ5r/mfQ32Mwg==  Propiedad  2019-09-15  2019-09-25   1.000000   
8  t0kNTwjTWOdat8Z87X7QLw==  Propiedad  2019-09-15  2019-09-25   1.682252   
9  fY4sOgwvFSS5Qz2M66X7/A==  Propiedad  2019-09-15  2019-09-25   1.000000   

   Total_Area  Covered_Area          Price Currency Price_Period  \
0  483.990998    437.290718  285718.897994      NaN      Mensual   
1   55.000000   

1. The operation took approximately 18.59 seconds to complete. This is the duration between measuring the time before and after performing the data loading operation.

2. The memory usage before the operation was 2102.53 megabytes (MB). This is the initial memory footprint of your Python process before loading the data.

3. The memory usage after the operation increased to 3361.19 megabytes (MB). This is the memory footprint of your Python process after loading the selected columns from the dataset.

4. The difference in memory usage (after - before) is 1258.66 megabytes (MB). This represents the additional memory consumed during the execution of the data loading operation

In [ ]:
df.head(5)

,ID,Ad_Type,Start_Date,End_Date,Created_On,Latitude,Longitude,Country_L1,Province_L2,City_L3,...,Bathrooms,Total_Area,Covered_Area,Price,Currency,Price_Period,Title,Description,Property_Type,Operation_Type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,2019-09-15,-26.815439,-65.312393,Argentina,Tucumán,Yerba Buena,...,1.682252,483.990998,437.290718,285718.897994,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,2019-09-15,-26.839469,-65.212790,Argentina,Tucumán,Mar del Plata,...,1.000000,55.000000,41.000000,285718.897994,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.424820,-64.181225,Argentina,Córdoba,Córdoba,...,1.000000,45.000000,45.000000,285718.897994,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.421242,-64.190798,Argentina,Córdoba,Córdoba,...,1.000000,48.000000,48.000000,285718.897994,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.426064,-64.180042,Argentina,Córdoba,Córdoba,...,1.000000,45.000000,45.000000,285718.897994,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal


###Optimize Data Types


We fine-tuned data types to maximize efficiency and minimize memory consumption.

In [80]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

df_optimize = pd.read_csv(file_path)
df_optimize.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
              'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
              'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
              'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']

df_optimize['Latitude'] = df_optimize['Latitude'].astype('float32')
df_optimize['Longitude'] = df_optimize['Longitude'].astype('float32')
df_optimize['Rooms'] = df_optimize['Rooms'].astype('float32')
df_optimize['Bedrooms'] = df_optimize['Bedrooms'].astype('float32')
df_optimize['Bathrooms'] = df_optimize['Bathrooms'].astype('float32')
df_optimize['Total_Area'] = df_optimize['Total_Area'].astype('float32')
df_optimize['Covered_Area'] = df_optimize['Covered_Area'].astype('float32')
df_optimize['Price'] = df_optimize['Price'].astype('float32')

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")


Execution Time: 19.082032918930054 seconds
Memory Usage Before: 5214.73828125 MB
Memory Usage After: 6385.328125 MB
Memory Usage Difference: 1170.58984375 MB


###Sampling

In this part we will implement sampling methodologies to extract meaningful insights from a subset of the dataset.

In [ ]:
sample_size=100000

In [ ]:
df_random_sample = pd.read_csv(file_path).sample(n=sample_size, random_state=42)

In [ ]:
print(df_random_sample.head(10))
print(df_random_sample.size)
print(df_random_sample.columns)
print(df_random_sample.shape)
print(df_random_sample.info())
# Perform operations
df_random_sample = df_random_sample.drop_duplicates()
result = df_random_sample.count()

df_random_sample.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
              'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
              'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
              'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']
# l5 and L6 are not clear, so drop them
null_counts = df_random_sample.isnull().sum()

df_random_sample = df_random_sample.drop(['L5', 'L6'], axis=1)

#Impute Categorical Values:
df_random_sample['City_L3'].fillna(df_random_sample['City_L3'].mode()[0], inplace=True)
df_random_sample['Neighbourhood_L4'].fillna(df_random_sample['Neighbourhood_L4'].mode()[0], inplace=True)
df_random_sample['Price_Period'].fillna(df_random_sample['Price_Period'].mode()[0], inplace=True)

#text
df_random_sample['Title'].fillna('', inplace=True)
df_random_sample['Description'].fillna('', inplace=True)

#Impute Numerical Values
df_random_sample['Latitude'].fillna(df_random_sample['Latitude'].mean(), inplace=True)
df_random_sample['Longitude'].fillna(df_random_sample['Longitude'].mean(), inplace=True)
df_random_sample['Rooms'].fillna(df_random_sample['Rooms'].mean(), inplace=True)
df_random_sample['Bedrooms'].fillna(df_random_sample['Bedrooms'].mean(), inplace=True)
df_random_sample['Bathrooms'].fillna(df_random_sample['Bathrooms'].mean(), inplace=True)
df_random_sample['Total_Area'].fillna(df_random_sample['Total_Area'].mean(), inplace=True)
df_random_sample['Covered_Area'].fillna(df_random_sample['Covered_Area'].mean(), inplace=True)
df_random_sample['Price'].fillna(df_random_sample['Price'].mean(), inplace=True)
df_random_sample.isnull().sum()

In [ ]:
df_percentage_sample = pd.read_csv(file_path).sample(frac=0.1, random_state=42)

In [ ]:
print(df_percentage_sample.head(10))
print(df_percentage_sample.size)
print(df_percentage_sample.columns)
print(df_percentage_sample.shape)
print(df_percentage_sample.info())
# Perform operations
df_percentage_sample = df_percentage_sample.drop_duplicates()
result = df_percentage_sample.count()

df_percentage_sample.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
              'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
              'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
              'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']
# l5 and L6 are not clear, so drop them
null_counts = df_percentage_sample.isnull().sum()

df_percentage_sample = df_percentage_sample.drop(['L5', 'L6'], axis=1)

#Impute Categorical Values:
df_percentage_sample['City_L3'].fillna(df_percentage_sample['City_L3'].mode()[0], inplace=True)
df_percentage_sample['Neighbourhood_L4'].fillna(df_percentage_sample['Neighbourhood_L4'].mode()[0], inplace=True)
df_percentage_sample['Price_Period'].fillna(df_percentage_sample['Price_Period'].mode()[0], inplace=True)

#text
df_percentage_sample['Title'].fillna('', inplace=True)
df_percentage_sample['Description'].fillna('', inplace=True)

#Impute Numerical Values
df_percentage_sample['Latitude'].fillna(df_percentage_sample['Latitude'].mean(), inplace=True)
df_percentage_sample['Longitude'].fillna(df_percentage_sample['Longitude'].mean(), inplace=True)
df_percentage_sample['Rooms'].fillna(df_percentage_sample['Rooms'].mean(), inplace=True)
df_percentage_sample['Bedrooms'].fillna(df_percentage_sample['Bedrooms'].mean(), inplace=True)
df_percentage_sample['Bathrooms'].fillna(df_percentage_sample['Bathrooms'].mean(), inplace=True)
df_percentage_sample['Total_Area'].fillna(df_percentage_sample['Total_Area'].mean(), inplace=True)
df_percentage_sample['Covered_Area'].fillna(df_percentage_sample['Covered_Area'].mean(), inplace=True)
df_percentage_sample['Price'].fillna(df_percentage_sample['Price'].mean(), inplace=True)
df_percentage_sample.isnull().sum()

                              id    ad_type  start_date    end_date  \
987231  r+MtF7isQjwjsQ7NCDUCvQ==  Propiedad  2020-02-22  2020-02-22   
79954   pVU3Oh0YjV9IUjzY58aAdQ==  Propiedad  2020-01-06  2020-01-19   
567130  x0DZ4TYdY+KfBI/oGkQSbQ==  Propiedad  2019-06-29  2019-06-30   
500891  gyhswObFx4BE2JPUPFEefw==  Propiedad  2020-03-15  2020-04-28   
55399   qwjqDYWeU1QUGkJ0oJ1cSw==  Propiedad  2019-12-27  2020-01-30   
135049  Af8XGFanHc2kp83OWdg6hw==  Propiedad  2019-11-24  2019-12-09   
733378  ScjgHK3/wLc/cQ/7K6l9dg==  Propiedad  2019-08-10  2019-09-11   
732057  uUZuv4SKb0wxiy6C5qbPaQ==  Propiedad  2019-08-10  2019-08-15   
51333   Ru48y7pvryjzJMI13+cO6w==  Propiedad  2019-03-04  2019-03-13   
731479  i9FGUFujM1exJIVIFPKChw==  Propiedad  2019-08-10  2019-08-15   

        created_on        lat        lon         l1  \
987231  2020-02-22 -37.986956 -57.573805  Argentina   
79954   2020-01-06 -34.577448 -58.424296  Argentina   
567130  2019-06-29 -34.610551 -58.444383  Argentina  

The DataFrame, comprising 100,000 entries across 23 columns, occupies approximately 15.3 megabytes in memory. The data types include float32 for numerical values and object for categorical or text-based values. Most columns exhibit complete data, with the exception of the "Currency" column, which has 94,581 non-null entries.

###Parallelize with Dask

Now, we will try Dask. Dask is a powerful Python library designed for parallel computing and handling larger-than-memory datasets seamlessly. It extends the capabilities of Pandas, NumPy, and other popular libraries, enabling users to scale their data processing tasks from a single machine to distributed clusters effortlessly.

In [64]:
import dask.dataframe as dd

ddf = dd.from_pandas(df, npartitions=4)

# Now, perform parallelized operations
mean_price = ddf['price'].mean()

# To compute the result
result = mean_price.compute()

In [65]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()
ddf = dd.from_pandas(df, npartitions=4)

# Now, perform parallelized operations
mean_price = ddf['price'].mean()

# To compute the result
result = mean_price.compute()
print(ddf.size)
print(ddf.columns)
print(ddf.shape)
print(ddf.info())
# Perform operations
ddf = ddf.drop_duplicates()
result = ddf.count()

ddf.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
              'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
              'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
              'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']
null_counts = ddf.isnull().sum()

ddf.isnull().sum()

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

dd.Scalar<size-ag..., dtype=int64>
Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')
(Delayed('int-1752c20c-a6c1-4466-8828-4aae3449ce31'), 25)
<class 'dask.dataframe.core.DataFrame'>
Columns: 25 entries, id to operation_type
dtypes: object(16), float64(9)None
Execution Time: 2.9968104362487793 seconds
Memory Usage Before: 7392.66015625 MB
Memory Usage After: 7583.38671875 MB
Memory Usage Difference: 190.7265625 MB


1. The code took approximately 2.58 seconds to execute. This is the duration between measuring the time before and after performing the Dask operations.

2. The memory usage before the Dask operations was 4127.57 megabytes (MB).

3. The memory usage after the Dask operations increased to 4433.63 megabytes (MB).

4. The difference in memory usage (after - before) is 306.07 megabytes (MB). This represents the additional memory consumed during the execution of the Dask operations.

###Use Chunking

In this last part we used chunking. Chunking is a strategy used to process large datasets by breaking them into smaller, more manageable pieces. This approach helps to avoid memory issues that might arise when trying to load or process an entire dataset at once.

In [66]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

chunk_size = 10000

# Create an empty DataFrame to store the processed chunks
processed_chunks = []

# Iterate over chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Remove duplicate rows
    chunk = chunk.drop_duplicates()

    # Remove empty rows (assuming empty rows have NaN values in all columns)
    chunk = chunk.dropna(how='all')

    chunk.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
                      'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
                      'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
                      'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']

    chunk = chunk.drop(['L5', 'L6'], axis=1)

    # Text
    chunk['Title'].fillna('', inplace=True)
    chunk['Description'].fillna('', inplace=True)

    # Impute Numerical Values
    chunk['Latitude'].fillna(chunk['Latitude'].mean(), inplace=True)
    chunk['Longitude'].fillna(chunk['Longitude'].mean(), inplace=True)
    chunk['Rooms'].fillna(chunk['Rooms'].mean(), inplace=True)
    chunk['Bedrooms'].fillna(chunk['Bedrooms'].mean(), inplace=True)
    chunk['Bathrooms'].fillna(chunk['Bathrooms'].mean(), inplace=True)
    chunk['Total_Area'].fillna(chunk['Total_Area'].mean(), inplace=True)
    chunk['Covered_Area'].fillna(chunk['Covered_Area'].mean(), inplace=True)
    chunk['Price'].fillna(chunk['Price'].mean(), inplace=True)

    # Append the processed chunk to the list
    processed_chunks.append(chunk)

# Concatenate the processed chunks into a final DataFrame
df_cleaned = pd.concat(processed_chunks, ignore_index=True)

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")


Execution Time: 28.923211574554443 seconds
Memory Usage Before: 7200.71875 MB
Memory Usage After: 8251.02734375 MB
Memory Usage Difference: 1050.30859375 MB


1. The code took approximately 23.36 seconds to execute. This is the duration between measuring the time before and after performing the data cleaning operations on the chunks.

2. The memory usage before the chunk processing was 4405.75 megabytes (MB). This is the initial memory footprint of your Python process before cleaning the data.

3. The memory usage after the chunk processing increased to 5080.66 megabytes (MB). This is the memory footprint of your Python process after cleaning and concatenating the chunks.

4. The difference in memory usage (after - before) is 674.91 megabytes (MB). This represents the additional memory consumed during the execution of the chunk processing and concatenation operations.


In [ ]:
df_cleaned.head()

,ID,Ad_Type,Start_Date,End_Date,Created_On,Latitude,Longitude,Country_L1,Province_L2,City_L3,...,Bathrooms,Total_Area,Covered_Area,Price,Currency,Price_Period,Title,Description,Property_Type,Operation_Type


##Comprehansive analysis

#### In Terms of Approach:
| Strategy | Time (in seconds) | Memory Usage (in Miga Byte) | Analysis|      
|----------|------|--------------|---------|
|Traditional(pandas)|35.2|1087|Uses pandas for data manipulation, takes longer time and higher memory usage.|
|Load Less Data|27.3|973.3|Optimizes by loading only necessary data, reducing both time and memory usage.|
|Optimize Data Types|19|1000.5|Optimizing data types contributes to a significant time reduction, but memory usage remains high.|
|sample (random)|23.1|3.1|Uses random sampling, significantly reduces memory usage, moderately improves time.|
|sample (percentage_sample)|22.3|549.8|Uses a percentage-based sample, which reduces memory usage and slightly improves time.|
|Parallelize with Dask|2.9|190.7|Utilizes Dask for parallel processing, resulting in a substantial reduction in both time and memory usage.|
|Chunking|33.3|674.91|Chunking the data shows a trade-off with higher time and lower memory usage compared to the traditional approach.|


**Analysis Summary:**


*   Traditional pandas approach is time-consuming with high memory usage.
*   Load Less Data reduces time but has a moderate impact on memory.

*   Optimizing Data Types significantly reduces time but has limited impact on memory.

*   Random Sampling and Percentage-based Sampling strike a balance between time and memory usage.

*   Parallelization with Dask offers a substantial reduction in both time and memory usage.

*   Chunking provides a trade-off, reducing memory usage at the cost of increased processing time.




####Pros and Cons
| Technique                        | Pros                                           | Cons                                                         |
| ---------------------------------|-----------------------------------------------|--------------------------------------------------------------|
| Traditional (pandas)             | - Familiar and widely used.                   | - High time consumption.                                      |
|                                  | - Comprehensive functionality.                 | - High memory usage for large datasets.                       |
| Load Less Data                   | - Reduced time by loading only necessary data. | - Limited scope for analysis on the entire dataset.           |
|                                  | - Moderate impact on memory.                   | - Potential information loss.                                  |
| Optimize Data Types              | - Significant reduction in processing time.   | - Limited impact on memory usage.                              |
|                                  | - Improved efficiency in memory usage.        | - May require careful handling of data type conversions.     |
| Sample (Random)                  | - Balanced approach to time and memory usage.  | - Potential for non-representative samples.                   |
|                                  | - Preserves random data distribution.         | - Limited control over sample characteristics.               |
| Sample (Percentage-based Sampling)| - Maintains a good balance between time and memory. | - Possible bias depending on the sampling technique.         |
|                                  | - Allows for controlled sampling.              | - May not capture rare events well.                           |
| Parallelize with Dask            | - Substantial reduction in both time and memory usage. | - Learning curve for Dask usage.                            |
|                                  | - Scalable for large datasets.                 | - Overhead associated with parallelization.                   |
| Chunking                         | - Reduced memory usage compared to traditional approach. | - Increased processing time due to chunking overhead.      |
|                                  | - Allows processing of large datasets in smaller, manageable chunks. | - Requires careful consideration of chunk size for optimal results. |


##Conclustion

In handling big data, various strategies present trade-offs between processing time and memory usage. Traditional approaches like pandas may struggle with large datasets, resulting in prolonged processing times and high memory consumption. Optimization techniques, sampling methods, and chunking offer intermediate solutions, balancing time and memory, but they come with their own limitations and considerations.

For substantial gains in efficiency, parallelization with tools like Dask emerges as a powerful solution, delivering significant reductions in both processing time and memory requirements. Despite the learning curve and associated overhead, the scalability of parallelization makes it a compelling choice for large-scale data processing. The most effective strategy depends on the specific analysis requirements, available computing resources, and acceptable trade-offs between time and memory in the context of handling extensive and intricate datasets.